# Data Explorer

<a href="https://colab.research.google.com/github/neurologic/Neurophysiology-Lab/blob/main/modules/earthworm-giant-fiber-cap/Data-Explorer_earthworm-giant-fiber-cap.ipynb" target="_blank" rel="noopener noreferrer"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>   

<a id="toc"></a>
# Table of Contents

- [Introduction](#intro)
- [Setup](#setup)
- [Get your bouts and trials here](#one)
- [CAP analysis](#two)


<a id="intro"></a>
# Giant Fiber System Activity

In this lab you measured the activity of the giant fiber system using extracellular differential electrodes. The activity that you will observe in your recording is referred to as a **complex action potential** (CAP). You will apply your knowledge from the previous two labs to interpret this signal. One of the predominant analysis frameworks you will use is comparing *trials* across *bouts*.

<a id="setup"></a>
# Setup

[toc](#toc)

## Import and define functions

In [ ]:
#@title {display-mode: "form" }

#@markdown Run this code cell to import packages and define functions 
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import ndimage
from scipy.signal import hilbert,medfilt,resample, find_peaks, unit_impulse
import seaborn as sns
from datetime import datetime,timezone,timedelta
pal = sns.color_palette(n_colors=15)
pal = pal.as_hex()
import matplotlib.pyplot as plt
import random

from pathlib import Path

from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from ipywidgets import widgets, interact, interactive
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## Mount Google Drive

In [ ]:
#@title {display-mode: "form" }

#@markdown Run this cell to mount your Google Drive.

from google.colab import drive
drive.mount('/content/drive')

print('Task completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## Import data 

Import data digitized with *Nidaq USB6211* and recorded using *Bonsai-rx* as a *.bin* file

> If you would like sample this Data Explorer, but do not have data, you can download examples for the following experiments using the linked shared file:
    - [CAP measured across different inter-electrode distances](https://drive.google.com/file/d/1Zo5Pyit_mnx1uHpcU1yjWX5mla8aDNx_/view?usp=sharing). If you are using this example file, the sample rate was 30000 on two channels (channel 0 was the nerve signal and channel 1 was the stimulus monitor). Bout times were: [[2,10],[10,20],[20,30],[30,45],[45,55],[55,70],[70,85],[85,100],[100,120]]. The inter-electrode distances were: 1,2,3,4,5,6,7,8 and then back to 1 cm for the last bout. The stimulus anode was 3cm from the closest measurement electrode in bout 0.
    - [CAP measured at different distances from the stimulating electrodes](https://drive.google.com/file/d/1YErnCGiCch1Bd1pNL1HP8t-JF1zNzc3r/view?usp=sharing). Bout times were: [[2,8],[17,25],[37,45]]. The distances between stimulation and recording electrodes were: 3,x,and x cm. 

In [ ]:
#@title {display-mode: "form" }

#@markdown Specify the file path 
#@markdown to your recorded data in the colab runtime (find the filepath in the colab file manager):

filepath = "full filepath goes here"  #@param 
# filepath = '/Volumes/Untitled/BIOL247/data/earthworm-giant-fiber-cap/diff_cv_0.bin'  #@param 
# filepath = '/Volumes/Untitled/BIOL247/data/earthworm-giant-fiber-cap/diff_distance_stimmon_2.bin'

#@markdown Specify the sampling rate and number of channels recorded.

sampling_rate = 30000 #@param
number_channels = 2 #@param

# downsample = False #@param
# newfs = 10000 #@param

#@markdown After you have filled out all form fields, 
#@markdown run this code cell to load the data. 

filepath = Path(filepath)

# No need to edit below this line
#################################
data = np.fromfile(Path(filepath), dtype = np.float64)
data = data.reshape(-1,number_channels)
data_dur = np.shape(data)[0]/sampling_rate
print('duration of recording was %0.2f seconds' %data_dur)

fs = sampling_rate
# if downsample:
#     # newfs = 10000 #downsample emg data
#     chunksize = int(sampling_rate/newfs)
#     data = data[0::chunksize,:]
#     fs = int(np.shape(data)[0]/data_dur)

time = np.linspace(0,data_dur,np.shape(data)[0])

print('Data upload completed at ' + str(datetime.now(timezone(-timedelta(hours=5)))))

## Plot raw data

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this code cell to plot the imported data. <br> 
#@markdown Use the range slider to scroll through the data in time.
#@markdown Use the channel slider to choose which channel to plot
#@markdown Be patient with the range refresh... the more data you are plotting the slower it will be. 

slider_xrange = widgets.FloatRangeSlider(
    min=0,
    max=data_dur,
    value=(0,1),
    step= 1,
    readout=True,
    continuous_update=False,
    description='Time Range (s)')
slider_xrange.layout.width = '600px'

slider_chan = widgets.IntSlider(
    min=0,
    max=number_channels-1,
    value=0,
    step= 1,
    continuous_update=False,
    description='channel')
slider_chan.layout.width = '300px'

# a function that will modify the xaxis range
def update_plot(x,chan):
    fig, ax = plt.subplots(figsize=(10,5),num=1); #specify figure number so that it does not keep creating new ones
    starti = int(x[0]*fs)
    stopi = int(x[1]*fs)
    ax.plot(time[starti:stopi], data[starti:stopi,chan])

w = interact(update_plot, x=slider_xrange, chan=slider_chan);

For a more extensive ***RAW*** Data Explorer than the one provided in the above figure, use the [DataExplorer.py](https://raw.githubusercontent.com/neurologic/Neurophysiology-Lab/main/howto/Data-Explorer.py) application found in the [howto section](https://neurologic.github.io/Neurophysiology-Lab/howto/Dash-Data-Explorer.html) of the course website.

<a id="one"></a>
# Part I. Get your bouts and trials here

The presentation of each stimulus marks the start of each ***trial*** in your experiment. Changes in stimulus parameters mark the start/end of different ***bouts*** in your experiment. 

Therefore, our first task in processing and analyzing data from the experiment is to figure out the trial times. You will do this using the same algorithm that you used to detect EOD events in the last lab. In this analysis, you will be processing the stimulus monitor signal directly.

## Define trial times

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this cell to create an interactive plot with a slider to scroll 
#@markdown through the signal on a selected trial
#@markdown and set an appropriate event detection threshold for trial onsets 
#@markdown (based on a positive-going level crossing of the signal). 

slider_xrange = widgets.FloatRangeSlider(
    min=0,
    max=data_dur,
    value=(0,1),
    step= 0.5,
    readout=True,
    continuous_update=False,
    description='Time Range (s)',
    style = {'description_width': '200px'})
slider_xrange.layout.width = '600px'

slider_yrange = widgets.FloatRangeSlider(
    min=np.min(stim)-0.5,
    max=np.max(stim)+0.5,
    value=[np.min(stim),np.max(stim)],
    step=0.05,
    continuous_update=False,
    readout=True,
    description='yrange',
    style = {'description_width': '200px'})
slider_yrange.layout.width = '600px'

select_channel = widgets.Select(
    options=np.arange(np.shape(data)[1]), # start with a single trial on a single bout... it will update when runs ; old: np.arange(len(trial_times)),
    value=0,
    #rows=10,
    description='Channel used to detect trials',
    style = {'description_width': '200px'},
    disabled=False
)

slider_threshold = widgets.FloatSlider(
    min=np.min(stim)-0.1,
    max=np.max(stim)+0.1,
    value=np.mean(stim)+np.std(stim)*2,
    step=0.05,
    continuous_update=False,
    readout=True,
    description='peak detection threshold',
    style = {'description_width': '200px'})
slider_threshold.layout.width = '600px'


def update_plot(chan_,thresh_,xrange,yrange):
    fig, ax = plt.subplots(figsize=(10,5),num=1); #specify figure number so that it does not keep creating new ones
    
    signal = data[:,chan_]
    
    # get the changes in bool value for a bool of signal greater than threshold
    threshold_crossings = np.diff(signal > thresh_, prepend=False)

    # get indices where threshold crossings are true
    tcross = np.argwhere(threshold_crossings)[:,0]

    # get a mask for only positive level crossings
    mask_ = [signal[t+1]-signal[t] > 0 for t in tcross]
    
    # trial times are positive level crossings
    trial_times = tcross[mask_]/fs

    starti = int(xrange[0]*fs)+1
    stopi = int(xrange[1]*fs)-1
    ax.plot(time[starti:stopi], signal[starti:stopi], color='black')
    
    # ax.plot(tmp,color='black')
    ax.hlines(thresh_, time[starti],time[stopi],linestyle='--',color='green')
    ax.scatter(trial_times,[thresh_]*len(trial_times),marker='^',s=300,color='purple',zorder=3)
    ax.set_ylim(yrange[0],yrange[1])
    ax.set_xlim(xrange[0],xrange[1])
    

    ax.xaxis.set_minor_locator(AutoMinorLocator(5))

           
    return trial_times

w_trials_ = interactive(update_plot, chan_=select_channel, thresh_=slider_threshold, xrange=slider_xrange, yrange=slider_yrange);
display(w_trials_)

In [ ]:
#@title {display-mode: "form"}

#@markdown Run this cell to finalize the list of trial times 
#@markdown after settling on a channel and threshold in the interactive plot. <br> 
#@markdown This stores the trial times in an array called 'trial_times'.
trial_times = w_trials_.result



## Define Bouts

In [ ]:
#@title {display-mode: "form"}

#@markdown Use the interactive plot of the stimulus monitor channel in the last section to determine 
#@markdown the start and stop time for each of the bouts in your experiment. 
#@markdown Specify the list of bout ranges as follows: [[start of bout 0, end of bout 0],[start 1, end 1],...]] <br>

bouts_list = [[2,8],[17,25],[37,45]] #@param
# bouts_list = [[2,10],[10,20],[20,30],[30,45],[45,55],[55,70],[70,85],[85,100],[100,120]]

#@markdown Then run this code cell to programatically define the list of bouts as 'bouts_list'.

<a id="two"></a>
# Part II. CAP analysis

## Visualize Trials

In [ ]:
#@title {display-mode:"form"}

#@markdown Run this code cell to create an interactive plot to  
#@markdown examine the CAP on individual trials for each bout.
#@markdown You can overlay multple channels by selecting more than one.
#@markdown You can overlay multiple trials by selecting more than one. 
#@markdown (To select more than one item from an option menu, press the control/command key 
#@markdown while mouse clicking or shift while using up/down arrows)

slider_xrange = widgets.FloatRangeSlider(
    min=-0.05,
    max=0.1,
    value=(-0.001,0.03),
    step=0.0005,
    continuous_update=False,
    readout=True,
    readout_format='.4f',
    description='xrange (s)'
)
slider_xrange.layout.width = '600px'

slider_yrange = widgets.FloatRangeSlider(
    min=-1,
    max=1, # normal range for earthworm experiments
    value=(-0.5,0.5),
    step=0.01,
    continuous_update=False,
    readout=True,
    description='yrange'
)
slider_yrange.layout.width = '600px'

# trials in bout 0 to start...
trials_t = trial_times[(trial_times>bouts_list[0][0]) & (trial_times<bouts_list[0][1])]

select_channels = widgets.SelectMultiple(
    options=np.arange(np.shape(data)[1]), # start with a single trial on a single bout... it will update when runs ,
    value=[0],
    #rows=10,
    description='Channels',
    disabled=False
)

select_bouts = widgets.Select(
    options=np.arange(len(bouts_list)), # start with a single trial on a single bout... it will update when runs ; old: np.arange(len(trial_times)),
    value=0,
    #rows=10,
    description='Bouts',
    disabled=False
)

select_trials = widgets.SelectMultiple(
    options=np.arange(len(trials_t)), # start with a single trial on a single bout... it will update when runs ,
    value=[0],
    #rows=10,
    description='Trials',
    disabled=False
)



def update_plot(chan_list,trial_list,bout_,yrange,xrange):
    fig, ax = plt.subplots(figsize=(10,6))# ,ncols=1, nrows=1); #specify figure number so that it does not keep creating new ones
 
    win_0 = int(xrange[0]*fs)
    win_1 = int(xrange[1]*fs)
    xtime = np.linspace(xrange[0],xrange[1],(win_1 - win_0))
    
    trials_t = trial_times[(trial_times>bouts_list[bout_][0]) & (trial_times<bouts_list[bout_][1])]
    trials_init_ = np.arange(len(trials_t))
    select_trials.options = trials_init_
    select_trials.value = trial_list
    
    channel_colors = ['purple','green','blue','orange']
    for chan_ in chan_list:
        this_chan = data[:,chan_]
        for trial_ in trial_list:
            if trial_ in trials_init_:
                t_ = trials_t[trial_]

                if ((int(fs*t_)+win_0)>0) & ((int(fs*t_)+win_1))<len(this_chan):
                    data_sweep = this_chan[(int(fs*t_)+win_0):(int(fs*t_)+win_1)]

                    ax.plot(xtime,data_sweep,color=channel_colors[chan_],linewidth=2,alpha=0.5)
    

    ax.set_ylim(yrange[0],yrange[1]);
    ax.set_xlabel('seconds')
    ax.vlines(0,yrange[0],yrange[1],color='black')

    
#     # Change major ticks to show every 20.
    # ax_pwm.xaxis.set_major_locator(MultipleLocator(5))
    # ax_pwm.yaxis.set_major_locator(MultipleLocator(5))

    # # Change minor ticks to show every 5. (20/4 = 5)
    # ax_mro.yaxis.set_minor_locator(AutoMinorLocator(10))
    ax.xaxis.set_minor_locator(AutoMinorLocator(10))
    # ax_pwm.yaxis.set_minor_locator(AutoMinorLocator(5))

#     # Turn grid on for both major and minor ticks and style minor slightly
# #     # differently.
    ax.grid(which='major', color='gray', linestyle='-')
    ax.grid(which='minor', color='gray', linestyle=':')
#     ax_pwm.grid(which='major', color='gray', linestyle='-')
#     ax_pwm.grid(which='minor', color='gray', linestyle=':')


w = interact(update_plot, chan_list=select_channels,trial_list=select_trials, bout_=select_bouts, 
             yrange=slider_yrange, xrange = slider_xrange);
# display(w)

## Visualize Trials Across Bouts

The following code cell will overlay the trial data from multiple channels, bouts, and trials. Each channel will be plotted on a different subplot. Each bout will be a different color. Specified trials will be averaged within each bout. 

In [ ]:
#@title {display-mode:"form"}

#@markdown Specify which channels you want to visualize.
channel_list = [0] #@param
#@markdown Specify which bouts you want to visualize. 
bouts_all = [0,1] #@param
#@markdown Specify which trials you want to visualize for each bout. 
#@markdown <br> Note that there must be a list of trials for every bout.
trials_all = [[0,1,2],[0,1,2]] #@param
#@markdown Specify the time range around each trial time that you want to include in the plot. 
xrange = [-0.001,0.01] #@param

#@markdown Now run this code cell to create a plot that shows the trial-averaged response on each bout

win_0 = int(xrange[0]*fs)
win_1 = int(xrange[1]*fs)
xtime = np.linspace(xrange[0],xrange[1],(win_1 - win_0))

fig, ax = plt.subplots(figsize=(10,6),nrows=len(channel_list),ncols=1)

for chan_ in channel_list:
    
    this_chan = data[:,chan_]
    
    for i,bout_ in enumerate(bouts_all):
        data_avg = []
        trial_list = trials_all[i]
        trials_t = trial_times[(trial_times>bouts_list[bout_][0]) & (trial_times<bouts_list[bout_][1])]
        for trial_ in trial_list:

            t_ = trials_t[trial_]


            if ((int(fs*t_)+win_0)>0) & ((int(fs*t_)+win_1))<len(this_chan):
                # sweep = data[(int(fs*t_)+win_0):(int(fs*t_)+win_1),mro_channel] 
                data_sweep = this_chan[(int(fs*t_)+win_0):(int(fs*t_)+win_1)]
                data_avg.append(data_sweep)
        data_avg = np.asarray(data_avg).T
        
        if len(channel_list)>1:
          ax[chan_].plot(xtime,np.mean(data_avg,1),linewidth=2,alpha=0.6,label = f'bout {bout_}')
        if len(channel_list)==1:
          ax.plot(xtime,np.mean(data_avg,1),linewidth=2,alpha=0.6,label = f'bout {bout_}')

    

    # ax.set_ylim(yrange[0],yrange[1]);
    ax.set_xlabel('seconds')
    ax.legend()
    # ax.vlines(0,yrange[0],yrange[1],color='green')
    ax.xaxis.set_major_locator(MultipleLocator(0.002))
    ax.xaxis.set_minor_locator(AutoMinorLocator(10))
    ax.grid(which='major', color='gray', linestyle='-')
    ax.grid(which='minor', color='gray', linestyle=':')

<hr> 
Written by Dr. Krista Perks for courses taught at Wesleyan University.

<a id="setup"></a>

<a id="one"></a>

<a id="two"></a>

<a id="three"></a>

<a id="four"></a>